#**Mapbiomas UNET Brazil**
This script is proposed to automate the entire mapbiomes deep learning flow


In [ ]:
from google.colab import auth
auth.authenticate_user()


In [ ]:
# Necessary API's install to notebook VM.
!pip install earthengine-api --upgrade
!pip install oauth2client 
!pip install tensorboardcolab
!pip install google-api-python-client 
!pip install pygeoj
import logging
import os
logging.getLogger('googleapicliet.discovery_cache').setLevel(logging.ERROR)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize()

In [ ]:
# Tensorflow setup.
import os
%tensorflow_version 1.x
%load_ext tensorboard
import tensorflow as tf

tf.enable_eager_execution()
print(tf.__version__)
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
# Folium setup.
import folium
print(folium.__version__)

# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

In [ ]:
import tensorflow as tf
# Specify names locations for outputs in Cloud Storage. 
# INSERT YOUR GCP BUCKET HERE:
VERSION = '2';
BUCKET = 'aquaculture_brazil'
FOLDER = 'unet-aquicultura-brazil_version2_2018'
TRAINING_BASE = 'training_patches_2018'
EVAL_BASE = 'eval_patches_2018'
MODEL_DIR = '/content/drive/My Drive/BRAZIL_UNETAquicultura/training_'+ VERSION +'/'

# Patch Exportation Configs
BUCKET_patch = 'mosaic-gee'
FOLDER_patch = 'allPatch'
FOLDER_classification = 'aquaculture'

# Specify inputs (Landsat bands) to the model and the response variable.
opticalBands = ['swir1', 'nir', 'red','NDVI','NDWI','NDSI']
#thermalBands = ['B10', 'B11']
BANDS = opticalBands #+ thermalBands
RESPONSE = 'cluster'
FEATURES = BANDS + [RESPONSE]

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 256
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

# Sizes of the training and evaluation datasets.
TRAIN_SIZE = 16000
EVAL_SIZE = 8000

# Specify model training parameters.
BATCH_SIZE = 16
EPOCHS = 10
BUFFER_SIZE = 2000
OPTIMIZER = 'SGD' 
LOSS = 'BinaryCrossentropy'
METRICS = ['RootMeanSquaredError','Hinge']

# Imagery

Gather and setup the imagery to use for inputs (predictors).  This is a three-year, cloud-free, Landsat 8 composite.  Display it in the notebook for a sanity check.

In [ ]:
# The image input data is a cloud-masked median composite.
mapbiomas4 = ee.Image('projects/samm/Mapbiomas5/Aquicultura/Clusterized/2018');
aquicultura = mapbiomas4.eq(1).unmask(0);
image = ee.Image('projects/samm/SAMM/Mosaic/2018').addBands(aquicultura)
#image = ee.Image('projects/mapbiomas-workspace/TRANSVERSAIS/ZONACOSTEIRA/Mosaic_ZC_2018_colecao_3_ZC')
mapid = image.getMapId({'bands': ['swir1', 'nir', 'red'], 'min': 30, 'max': 150})
map = folium.Map(location=[-5.11, -36.65])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Planet',
    overlay=True,
    name='Mosaic composite',
  ).add_to(map)
map

Prepare the response (what we want to predict).  This is impervious surface area (in fraction of a pixel) from the 2016 NLCD dataset.  Display to check.

In [ ]:
mapid = image.select(RESPONSE).getMapId({'min': 0, 'max': 1})
map = folium.Map(location=[-5.11, -36.65])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='nlcd impervious',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

Stack the 2D images (Landsat composite and NLCD impervious surface) to create a single image from which samples can be taken.  Convert the image into an array image in which each pixel stores 256x256 patches of pixels for each band.  This is a key step that bears emphasis: to export training patches, convert a multi-band image to [an array image](https://developers.google.com/earth-engine/arrays_array_images#array-images) using [`neighborhoodToArray()`](https://developers.google.com/earth-engine/api_docs#eeimageneighborhoodtoarray), then sample the image at points.

In [ ]:
featureStack = ee.Image.cat([
  image.select(BANDS).unmask(0),
  image.select(RESPONSE).unmask(0)
]).float()

list = ee.List.repeat(1, KERNEL_SIZE)
lists = ee.List.repeat(list, KERNEL_SIZE)
kernel = ee.Kernel.fixed(KERNEL_SIZE, KERNEL_SIZE, lists)

arrays = featureStack.neighborhoodToArray(kernel)

Use some pre-made geometries to sample the stack in strategic locations.  Specifically, these are hand-made polygons in which to take the 256x256 samples.  Display the sampling polygons on a map, red for training polygons, blue for evaluation.

# Sampling

The mapped data look reasonable so take a sample from each polygon and merge the results into a single export.  The key step is sampling the array image at points, to get all the pixels in a 256x256 neighborhood at each point.  It's worth noting that to build the training and testing data for the FCNN, you export a single TFRecord file that contains patches of pixel values in each record.  You do NOT need to export each training/testing patch to a different image.  Since each record potentially contains a lot of data (especially with big patches or many input bands), some manual sharding of the computation is necessary to avoid the `computed value too large` error.  Specifically, the following code takes multiple (smaller) samples within each geometry, merging the results to get a single export.

In [ ]:
trainingPolys = ee.FeatureCollection('projects/samm/Mapbiomas5/Aquicultura/Geometries/train_2018')
evalPolys = ee.FeatureCollection('projects/samm/Mapbiomas5/Aquicultura/Geometries/test_2018')
print(trainingPolys.size().getInfo())

polyImage = ee.Image(0).byte().paint(trainingPolys, 1).paint(evalPolys, 2)
polyImage = polyImage.updateMask(polyImage)

mapid = polyImage.getMapId({'min': 1, 'max': 2, 'palette': ['red', 'blue']})
map = folium.Map(location=[-1.3621, -45.2738], zoom_start=5)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='training polygons',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [ ]:
# Convert the feature collections to lists for iteration.
trainingPolysList = trainingPolys.toList(trainingPolys.size())
evalPolysList = evalPolys.toList(evalPolys.size())
# These numbers determined experimentally.
n = 50 # Number of shards in each polygon.
N = 2000 # Total sample size in each polygon.

#Add some generalism
TRAIN_SIZE = trainingPolys.size().getInfo()*N
EVAL_SIZE = evalPolys.size().getInfo()*N
print('TRAIN:'+str(TRAIN_SIZE))
print('EVAL:'+str(EVAL_SIZE))

# Export all the training data (in many pieces), with one task 
# per geometry.
for g in range(trainingPolys.size().getInfo()):
  geomSample = ee.FeatureCollection([])
  for i in range(n):
    sample = arrays.sample(
      region = ee.Feature(trainingPolysList.get(g)).geometry(), 
      scale = 30, 
      numPixels = N / n, # Size of the shard.
      seed = i,
      tileScale = 8
    )
    geomSample = geomSample.merge(sample)
  
  desc = TRAINING_BASE + '_g' + str(g)
  task = ee.batch.Export.table.toCloudStorage(
    collection = geomSample,
    description = desc, 
    bucket = BUCKET, 
    fileNamePrefix = FOLDER + '/' + desc,
    fileFormat = 'TFRecord',
    selectors = BANDS + [RESPONSE]
  )
  task.start()

# Export all the evaluation data.
for g in range(evalPolys.size().getInfo()):
  geomSample = ee.FeatureCollection([])
  for i in range(n):
    sample = arrays.sample(
      region = ee.Feature(evalPolysList.get(g)).geometry(), 
      scale = 30, 
      numPixels = N / n,
      seed = i,
      tileScale = 8
    )
    geomSample = geomSample.merge(sample)
  
  desc = EVAL_BASE + '_g' + str(g)
  task = ee.batch.Export.table.toCloudStorage(
    collection = geomSample,
    description = desc, 
    bucket = BUCKET, 
    fileNamePrefix = FOLDER + '/' + desc,
    fileFormat = 'TFRecord',
    selectors = BANDS + [RESPONSE]
  )
  task.start()

# Training data

Load the data exported from Earth Engine into a `tf.data.Dataset`.  The following are helper functions for that.

In [ ]:
def parse_tfrecord(example_proto):
  """The parsing function.
  Read a serialized example into the structure defined by FEATURES_DICT.
  Args:
    example_proto: a serialized Example.
  Returns: 
    A dictionary of tensors, keyed by feature name.
  """
  return tf.io.parse_single_example(example_proto, FEATURES_DICT)


def to_tuple(inputs):
  """Function to convert a dictionary of tensors to a tuple of (inputs, outputs).
  Turn the tensors returned by parse_tfrecord into a stack in HWC shape.
  Args:
    inputs: A dictionary of tensors, keyed by feature name.
  Returns: 
    A dtuple of (inputs, outputs).
  """
  inputsList = [inputs.get(key) for key in FEATURES]
  stacked = tf.stack(inputsList, axis=0)
  # Convert from CHW to HWC
  stacked = tf.transpose(stacked, [1, 2, 0])
  return stacked[:,:,:len(BANDS)], stacked[:,:,len(BANDS):]


def get_dataset(pattern):
  """Function to read, parse and format to tuple a set of input tfrecord files.
  Get all the files matching the pattern, parse and convert to tuple.
  Args:
    pattern: A file pattern to match in a Cloud Storage bucket.
  Returns: 
    A tf.data.Dataset
  """
  glob = tf.gfile.Glob(pattern)
  dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
  dataset = dataset.map(parse_tfrecord, num_parallel_calls=5)
  dataset = dataset.map(to_tuple, num_parallel_calls=5)
  return dataset

In [ ]:
def get_training_dataset():
	"""Get the preprocessed training dataset
  Returns: 
    A tf.data.Dataset of training data.
  """
	glob = 'gs://aquaculture_brazil/unet-aquicultura-brazil_version2_2018/training_patches_2018 + '*'
	print(glob)
	dataset = get_dataset(glob)
	dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
	return dataset

training = get_training_dataset()

print(iter(training.take(1)).next())

# Evaluation data

Now do the same thing to get an evaluation dataset.  Note that unlike the training dataset, the evaluation dataset has a batch size of 1, is not repeated and is not shuffled.

In [ ]:
def get_eval_dataset():
	glob = 'gs://' + BUCKET + '/' + FOLDER + '/' + EVAL_BASE + '*'
	dataset = get_dataset(glob)
	dataset = dataset.batch(1).repeat()
	return dataset

evaluation = get_eval_dataset()

# Model

Here we use the Keras implementation of the U-Net model as found [in the TensorFlow examples](https://github.com/tensorflow/models/blob/master/samples/outreach/blogs/segmentation_blogpost/image_segmentation.ipynb).  The U-Net model takes 256x256 pixel patches as input and outputs per-pixel class probability, label or a continuous output.  We can implement the model essentially unmodified, but will use mean squared error loss on the sigmoidal output since we are treating this as a regression problem, rather than a classification problem.  Since impervious surface fraction is constrained to [0,1], with many values close to zero or one, a saturating activation function is suitable here.

In [ ]:

from tensorflow.python.keras import layers
from tensorflow.python.keras import losses
from tensorflow.python.keras import models
from tensorflow.python.keras import metrics
from tensorflow.python.keras import optimizers

def conv_block(input_tensor, num_filters):
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('relu')(encoder)
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('relu')(encoder)
	return encoder

def encoder_block(input_tensor, num_filters):
	encoder = conv_block(input_tensor, num_filters)
	encoder_pool = layers.MaxPooling2D((2, 2), strides=(2, 2))(encoder)
	return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
	decoder = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
	decoder = layers.concatenate([concat_tensor, decoder], axis=-1)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	return decoder

def get_model():
	inputs = layers.Input(shape=[None, None, len(BANDS)]) # 256
	encoder0_pool, encoder0 = encoder_block(inputs, 64) # 128
	encoder1_pool, encoder1 = encoder_block(encoder0_pool, 128) # 64
	encoder2_pool, encoder2 = encoder_block(encoder1_pool, 256) # 32
	encoder3_pool, encoder3 = encoder_block(encoder2_pool, 512) # 16
	center = conv_block(encoder3_pool, 1024) # center
	decoder4 = decoder_block(center, encoder3, 512) # 16
	decoder3 = decoder_block(decoder4, encoder2, 256) # 32
	decoder2 = decoder_block(decoder3, encoder1, 128) # 64
	decoder1 = decoder_block(decoder2, encoder0, 64) # 128
	#outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(decoder1)
	#dropout = tf.keras.layers.Dropout(decoder1, rate=0.5, name="dropout")
	dropout = layers.Dropout(0.2, name="dropout", noise_shape=None, seed=None)(decoder1)
	outputs = layers.Conv2D(1, (1, 1),  activation=tf.nn.sigmoid, padding='same', kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d())(dropout)
	model = models.Model(inputs=[inputs], outputs=[outputs])
	#loss = twoclass_cost(output, labels_clipped)
	optimizer = tf.contrib.opt.NadamOptimizer(0.05, name='optimizer')
	model.compile(
		optimizer=optimizer, 
		loss=losses.get(LOSS),
		metrics=[metrics.get(metric) for metric in METRICS])

	return model

In [ ]:
#Tensorflow Image Prediction sampling
import keras
from PIL import Image
import io
import numpy as np
import tensorflow as t

class ImageHistory(keras.callbacks.Callback):
    
    def __init__(self, tensor_board_dir, data, last_step=0, draw_interval=100):
        self.data = data
        self.last_step = last_step
        self.draw_interval = draw_interval
        self.tensor_board_dir = tensor_board_dir
        
    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        images = []
        labels = []
        for item in self.data:
            image_data = item[0]
            label_data = item[1]
            y_pred = self.model.predict(image_data)
            images.append(y_pred)
            labels.append(label_data)
        image_data = np.concatenate(images,axis=2)
        label_data = np.concatenate(labels,axis=2)
        data = np.concatenate((image_data,label_data), axis=1)
        self.saveToTensorBoard(data, 'epoch', epoch)
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        if batch % self.draw_interval == 0:
            images = []
            labels = []
            for item in self.data:
                image_data = item[0]
                label_data = item[1]
                y_pred = self.model.predict(image_data)
                images.append(y_pred)
                labels.append(label_data)
            image_data = np.concatenate(images,axis=2)
            label_data = np.concatenate(labels,axis=2)
            data = np.concatenate((image_data,label_data), axis=1)
            self.last_step += 1
            self.saveToTensorBoard(data, 'batch', self.last_step*self.draw_interval)
        return
    
    def make_image(self, npyfile):
        """
        Convert an numpy representation image to Image protobuf.
        taken and updated from https://github.com/lanpa/tensorboard-pytorch/
        """
        height, width, channel = npyfile.shape
        image = Image.frombytes('L',(width,height), npyfile.tobytes())
        output = io.BytesIO()
        image.save(output, format='PNG')
        image_string = output.getvalue()
        output.close()
        return tf.Summary.Image(height=height, width=width, colorspace=channel,
                             encoded_image_string=image_string)
    
    def saveToTensorBoard(self, npyfile, tag, epoch):
        data = npyfile[0,:,:,:]
        image = (((data - data.min()) * 255) / (data.max() - data.min())).astype(np.uint8)
        image = self.make_image(image)
        summary = tf.Summary(value=[tf.Summary.Value(tag=tag, image=image)])
        writer = tf.summary.FileWriter(self.tensor_board_dir)
        writer.add_summary(summary, epoch)
        writer.close()       

# Training the model

You train a Keras model by calling `.fit()` on it.  Here we're going to train for 10 epochs, which is suitable for demonstration purposes.  For production use, you probably want to optimize this parameter, for example through [hyperparamter tuning](https://cloud.google.com/ml-engine/docs/tensorflow/using-hyperparameter-tuning).

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
m = get_model()
#print(m.summary())

MODEL_DIR = '/content/drive/My Drive/BRAZIL_UNETAquicultura/training_2/cp-0074.ckpt' #2018+2005
m.load_weights(MODEL_DIR)

#loss, acc = m.evaluate(test_images, test_labels)
#print("Restored model, accuracy: {:5.2f}%".format(100*acc))

In [ ]:
import keras.backend as K

%tensorboard --logdir .logs
# define TensorBoard directory
tb_dir = '/content/logs/'


In [ ]:
checkpoint_path = "/content/drive/My Drive/BRAZIL_UNETAquicultura/training_" + VERSION + "/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True, period=2)
m.save_weights(checkpoint_path.format(epoch=0))

image_history = ImageHistory(tensor_board_dir=tb_dir,
        data=training, last_step=0, draw_interval=100)

#logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard("/content/logs", histogram_freq=1)

with tf.device('/device:GPU:0'):
  m.fit(
      x=training, 
      epochs=80, 
      initial_epoch=74, # REMEBER TO CHANGE THIS INITIAL EPOCH PARAM, WHEN ANOTHER MODEL HAS BEEN LOADED
      steps_per_epoch=int(TRAIN_SIZE / BATCH_SIZE), 
      validation_data=evaluation,
      validation_steps=EVAL_SIZE,
      callbacks = [cp_callback,tensorboard_callback])#image_history])
      #callbacks = [cp_callback,TensorBoardColabCallback(tbc)])


In [ ]:
# Load a trained model. 50 epochs. 25 hours. Final RMSE ~0.08.
#MODEL_DIR = '/content/drive/My Drive/FCNN/training/cp.ckpt'
#m = tf.contrib.saved_model.load_keras_model(MODEL_DIR)
#m.summary()

In [ ]:
def doExport(out_image_base,index_in, kernel_buffer, bjregion):
  """Run the image export task.  Block until complete.
  """
  index = index_in
  image = ee.Image('projects/samm/SAMM/Mosaic/'+str(index))
  #image = ee.Image('projects/mapbiomas-workspace/TRANSVERSAIS/ZONACOSTEIRA/Mosaic_ZC_'+str(index)+'_colecao_3_ZC')
  out_image_base2 = out_image_base+'_'+str(index_in)
  filesList = !gsutil ls 'gs://'{BUCKET_patch}'/'{FOLDER_patch}'/'{out_image_base2}'*'
  exportFilesList = [s for s in filesList if out_image_base in s]
  if(len(exportFilesList) > 1):
    print('Image Already Exported')
    return None
  print("Exporting..")
  
  task = ee.batch.Export.image.toCloudStorage(
    image = image.select(BANDS).toFloat(), 
    description = out_image_base+'_'+str(index), 
    bucket = BUCKET_patch, 
    fileNamePrefix = FOLDER_patch + '/' + out_image_base+'_'+str(index), 
    region = bjregion, 
    scale = 30, 
    fileFormat = 'TFRecord', 
    maxPixels = 1e13,
    formatOptions = { 
      'patchDimensions': KERNEL_SHAPE,
      'kernelSize': kernel_buffer,
      'compressed': True,
      'maxFileSize': 104857600
    }
  )
  task.start()

  # Block until the task completes.
  print('Running image export to Cloud Storage...')
  import time
  #while task.active():
  time.sleep(5)

  # Error condition
  if task.status()['state'] != 'COMPLETED':
    print('Error with image export.')
  else:
    print('Image export completed.')

In [ ]:
def doPrediction(out_image_base,index_in, user_folder, kernel_buffer, region):
  """Perform inference on exported imagery, upload to Earth Engine.
  """
  #BUGFIX YEAR 
  out_image_base = out_image_base+'_'+str(index_in)
  #check if this file already exists on bucket
  # Get a list of all the files in the output bucket.
  filesList = !gsutil ls 'gs://'{BUCKET_patch}'/'{FOLDER_patch}'/'{out_image_base}'*'
  # Get only the files generated by the image export.
  exportFilesList = [s for s in filesList if out_image_base in s]

  # Get the list of image files and the JSON mixer file.
  imageFilesList = []
  jsonFile = None
  for f in exportFilesList:
    if f.endswith('.tfrecord.gz'):
      imageFilesList.append(f)
    elif f.endswith('.json'):
      jsonFile = f

  # Make sure the files are in the right order.
  imageFilesList.sort()
  
  
  predictioned_file =  !gsutil ls 'gs://'{BUCKET_patch}'/'{FOLDER_classification}'/'{out_image_base}'.TFRecord'
  out_image_asset = user_folder + '/' + out_image_base
  out_image_file = 'gs://' + BUCKET_patch + '/' + FOLDER_classification + '/' + out_image_base + '.TFRecord'

  if predictioned_file[0] == 'gs://'+ BUCKET_patch+'/'+FOLDER_classification+'/'+out_image_base+'.TFRecord':
    print('classification was already on Bucket')
    try:
      gee_asset = ee.Image(out_image_asset).bandNames().getInfo()
      if len(gee_asset) > 0:
        print('classification was already on GEE')
        return None 
    except ee.EEException:
      print('classification has not been uploaded')
      print('!earthengine --no-use_cloud_api upload image --asset_id='+out_image_asset+' '+out_image_file+' '+jsonFile)
      !earthengine --no-use_cloud_api upload image --asset_id={out_image_asset} {out_image_file} {jsonFile}
    return None 
    
  import json
  # Load the contents of the mixer file to a JSON object.
  jsonText = !gsutil cat {jsonFile}
  # Get a single string w/ newlines from the IPython.utils.text.SList
  mixer = json.loads(jsonText.nlstr)
  #pprint(mixer)
  patches = mixer['totalPatches']
  
  # Get set up for prediction.
  x_buffer = int(kernel_buffer[0] / 2)
  y_buffer = int(kernel_buffer[1] / 2)

  buffered_shape = [
      KERNEL_SHAPE[0] + kernel_buffer[0],
      KERNEL_SHAPE[1] + kernel_buffer[1]]

  imageColumns = [
    tf.FixedLenFeature(shape=buffered_shape, dtype=tf.float32) 
      for k in BANDS
  ]

  imageFeaturesDict = dict(zip(BANDS, imageColumns))

  def parse_image(example_proto):
    return tf.parse_single_example(example_proto, imageFeaturesDict)

  def toTupleImage(dict):
    inputsList = [dict.get(key) for key in BANDS]
    stacked = tf.stack(inputsList, axis=0)
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked
  
  # Create a dataset from the TFRecord file(s) in Cloud Storage.
  imageDataset = tf.data.TFRecordDataset(imageFilesList, compression_type='GZIP')
  imageDataset = imageDataset.map(parse_image, num_parallel_calls=5)
  imageDataset = imageDataset.map(toTupleImage).batch(1)
  
  # Perform inference.
  print('Running predictions...')
  with tf.device('/device:GPU:0'):
    predictions = m.predict(imageDataset, steps=patches, verbose=1)
  # print(predictions[0])

  #print('Writing predictions...')
  out_image_file = 'gs://' + BUCKET_patch + '/' + FOLDER_classification + '/' + out_image_base + '.TFRecord'
  writer = tf.python_io.TFRecordWriter(out_image_file)
  patches = 0
  for predictionPatch in predictions:
    #print('Writing patch ' + str(patches) + '...')
    predictionPatch = predictionPatch[
        x_buffer:x_buffer+KERNEL_SIZE, y_buffer:y_buffer+KERNEL_SIZE]

    # Create an example.
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'impervious': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=predictionPatch.flatten()))
        }
      )
    )
    # Write the example.
    writer.write(example.SerializeToString())
    patches += 1

  writer.close()

  # Start the upload.
  out_image_asset = user_folder + '/' + out_image_base
  !earthengine --no-use_cloud_api upload image --asset_id={out_image_asset} {out_image_file} {jsonFile}

Now there's all the code needed to run the prediction pipeline, all that remains is to specify the output region in which to do the prediction, the names of the output files, where to put them, and the shape of the outputs.  In terms of the shape, the model is trained on 256x256 patches, but can work (in theory) on any patch that's big enough with even dimensions ([reference](https://www.cv-foundation.org/openaccess/content_cvpr_2015/papers/Long_Fully_Convolutional_Networks_2015_CVPR_paper.pdf)).  Because of tile boundary artifacts, give the model slightly larger patches for prediction, then clip out the middle 256x256 patch.  This is controlled with a kernel buffer, half the size of which will extend beyond the kernel buffer.  For example, specifying a 128x128 kernel will append 64 pixels on each side of the patch, to ensure that the pixels in the output are taken from inputs completely covered by the kernel.  

In [ ]:
import pygeoj
kernel_buffer = [256, 256]
image_base_name = 'UNET_grid_'
user_folder = 'projects/mapbiomas-workspace/TRANSVERSAIS/AQUICULTURA_5' # INSERT YOUR FOLDER HERE.
# Base file name to use for TFRecord files and assets.
grid = pygeoj.load("/content/drive/My Drive/BRAZIL_UNETAquicultura/Grid_2deg_ZC_ovr.geojson")
print("Total Features on GRID")
print(len(grid))
print(grid.get_feature(1).geometry.coordinates)

bj_region =  None;

In [ ]:
# Run the export.
for region in grid:
    region_id = region.properties['id']
    if int(region_id) > 0:
      print('Region:')
      print(region_id)
      for y in range(1985, 2000):
          doExport(image_base_name+str(region_id)+str('_' + VERSION),y, kernel_buffer, region.geometry.coordinates)

In [ ]:

for  y in range(1991, 2018):
    for region in grid:
      region_id = region.properties['id']
      print(region_id)
      if int(region_id) > 0 :
        doPrediction(image_base_name+str(int(region_id))+str('_' + VERSION),y, user_folder, kernel_buffer, region.geometry.coordinates)


In [ ]:
imageCollection = ee.ImageCollection('projects/nexgenmap/PSScene4Band')
size = 50
index = 21
imageCollection = imageCollection.filterBounds(bj_region).toList(size)
imageInp = ee.Image(imageCollection.get(index))
imageArea = imageInp.mask().int().mask().int().reduceToVectors(ee.Reducer.max(), bj_region, 3, None, True, None, None, None, True, 1e13, 1, True).filterMetadata('label','equals',1)
mapid2 = imageInp.getMapId({'bands': ['R', 'G', 'B'], 'min': 2000, 'max': 11000})

out_image = ee.Image(user_folder + '/' + bj_image_base)
mapid = out_image.getMapId({'min': 0, 'max': 1})
map = folium.Map(location=[-1.3621, -45.2738])
folium.TileLayer(
    tiles=EE_TILES.format(**mapid),
    attr='Google Earth Engine',
    overlay=True,
    name='Predict',
  ).add_to(map)
folium.TileLayer(
    tiles=EE_TILES.format(**mapid2),
    attr='Google Earth Engine',
    overlay=True,
    name='Input',
  ).add_to(map)
map.add_child(folium.LayerControl())
map